In [1]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

In [2]:
import numpy as np
import pandas as pd

In [3]:
import datetime as dt

# Reflect Tables into SQLAlchemy ORM

In [4]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [5]:
engine = create_engine("sqlite:///hawaii.sqlite")
inspector = inspect(engine)
inspector.get_table_names()

['measurement', 'station']

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
# reflect the tables
Base.prepare(engine, reflect=True)

In [ ]:
# We can view all of the classes that automap found
Base.classes.keys()

In [7]:
# Save references to each table
Measurement = Base.classes.measurement
Station = Base.classes.station
inspector = inspect(engine)
# columns = inspector.get_columns('Station')
# for c in columns:
#     print(c['name'], c["type"])
columns = inspector.get_columns('Measurement')
for c in columns:
    print(c['name'], c["type"])
# engine.execute('SELECT * FROM Measurement LIMIT 5').fetchall()

NameError: name 'Base' is not defined

In [ ]:
# Create our session (link) from Python to the DB
session = Session(engine)

# Exploratory Climate Analysis

In [ ]:
# Design a query to retrieve the last 12 months of precipitation data and plot the results. 
#Starting from the last data point in the database. 
cur_year = dt.date(2017,8,23)

# Calculate the date one year from the last date in data set
prev_year = cur_year - dt.timedelta(days = 365)
prev_year
# Perform a query to retrieve the data and precipitation scores
results = session.query(Measurement.date,Measurement.prcp).filter(Measurement.date>=prev_year).all()
# Save the query results as a Pandas DataFrame and set the index to the date column
df = pd.DataFrame(results, columns = ['date','precipitation'])
# Sort the dataframe by date
df.set_index(df['date'],inplace = True)
# print(df.to_string(index=False))
df = df.sort_index()
df
# Use Pandas Plotting with Matplotlib to plot the data
df.plot(rot='vertical',fontsize = 10)

In [ ]:
# Use Pandas to calcualte the summary statistics for the precipitation data
df.describe()

In [ ]:
# How many stations are available in this dataset?
session.query(func.count(Station.station)).all()

In [ ]:
# What are the most active stations?
# List the stations and the counts in descending order.
session.query(Measurement.station,func.count(Measurement.station)).\
    group_by(Measurement.station).order_by(func.count(Measurement.station).desc()).all()

In [ ]:
# Using the station id from the previous query, calculate the lowest temperature recorded, 
# highest temperature recorded, and average temperature most active station?
session.query(func.min(Measurement.tobs), func.max(Measurement.tobs),func.avg(Measurement.tobs)).\
              filter(Measurement.station =='USC00519281').all()

In [ ]:
# Choose the station with the highest number of temperature observations.
# Query the last 12 months of temperature observation data for this station and plot the results as a histogram
results = session.query(Measurement.tobs).\
filter(Measurement.station =='USC00519281').\
filter(Measurement.date >=prev_year).all()
df_station = pd.DataFrame(results,columns = ['tobs'])
df_station.plot.hist(bins=20)
plt.tight_layout()

In [ ]:
# Write a function called `calc_temps` that will accept start date and end date in the format '%Y-%m-%d' 
# and return the minimum, average, and maximum temperatures for that range of dates
def calc_temps(st_date, end_date):
    minimum = session.query(func.min(Measurement.tobs)).\
    filter(func.strftime('%Y-%m-%d',Measurement.date)>= st_date,func.strftime('%Y-%m-%d',Measurement.date)<= end_date).all()
    maximum = session.query(func.max(Measurement.tobs)).\
    filter(func.strftime('%Y-%m-%d',Measurement.date)>= st_date,func.strftime('%Y-%m-%d',Measurement.date)<= end_date).all()
    average = session.query(func.avg(Measurement.tobs)).\
    filter(func.strftime('%Y-%m-%d',Measurement.date)>= st_date,func.strftime('%Y-%m-%d',Measurement.date)<= end_date).all()
    length = len(session.query(Measurement.tobs).\
    filter(func.strftime('%Y-%m-%d',Measurement.date)>= st_date,func.strftime('%Y-%m-%d',Measurement.date)<= end_date).all())
    return minimum, maximum, average,length
calc_temps('2017-06-23', '2017-07-23')

# Challenge

In [ ]:
session.query(func.count(Measurement.date)).all()

In [ ]:
session.query(Measurement.date).order_by(Measurement.date.desc()).first()

In [ ]:
test = session.query(Measurement.date).\
    filter(Measurement.date>'2016-08-23', Measurement.date<'2017-06-23').order_by(Measurement.date).all()
len(test)

In [ ]:
import datetime as dt
print (dt.date.today())

In [ ]:
print(dt.date(2017,10,8))

In [ ]:
d = dt.date(input("what is the date")).strftime('%Y-%m-%d')

In [ ]:
date_str = input(f"What is the date:")

In [ ]:
date_str

In [ ]:
session.query(Measurement.date).\
filter(func.strftime('%Y-%m-%d',Measurement.date) == date_str).all()